In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

gameIDs = []
season_games = pd.read_csv('netball.csv')
dfdup = season_games[season_games.duplicated()]
df_dropped = season_games.drop_duplicates()
df_dropped = df_dropped.reset_index()
for index, row in df_dropped.iterrows():
    gameIDs.append(str(index))
df_dropped['GameIDs'] = gameIDs

In [ ]:
datetime  = [row['Date'].split()[-2:]  for index,row in df_dropped.iterrows()]
for  index,row in df_dropped.iterrows():
  year = int(row['Season'])
  datetime[index].append(year)

In [ ]:
def months_as_num(date_list):
    months = []
    for i in range(len(date_list)):
        if date_list[i][1][:3].lower() == 'jan':
            months.append(1)
        if date_list[i][1][:3].lower() == 'feb':
            months.append(2)
        if date_list[i][1][:3].lower() == 'mar':
            months.append(3)
        if date_list[i][1][:3].lower() == 'apr':
            months.append(4)
        if date_list[i][1][:3].lower() == 'may':
            months.append(5)
        if date_list[i][1][:3].lower() == 'jun':
            months.append(6)
        if date_list[i][1][:3].lower() == 'jul':
            months.append(7)
        if date_list[i][1][:3].lower() == 'aug':
            months.append(8)
        if date_list[i][1][:3].lower() == 'sep':
            months.append(9)
        if date_list[i][1][:3].lower() == 'oct':
            months.append(10)
        if date_list[i][1][:3].lower() == 'nov':
            months.append(11)
        if date_list[i][1][:3].lower() == 'dec':
            months.append(12)
    return months
def list_as_nums(date_list,col):
    nums = []
    for i in range(len(date_list)):
        nums.append(int(date_list[i][col]))
    return nums

In [ ]:
days = []
years = []
months = []
days = list_as_nums(datetime,0)
years = list_as_nums(datetime,2)
months = months_as_num(datetime)
datedf = pd.to_datetime({'year':years,'month':months,'day':days})


In [ ]:
def drops_cols(df,cols):
    return df.drop(columns=cols)
cols_to_drop = ['Date']
netball_df = drops_cols(df_dropped,cols_to_drop)
netball_df['Date'] = datedf
netball_df.head()

In [ ]:
def expected_win_prob(elo_h,elo_a,home_court):
    home_elo_adjusted = elo_h + home_court
    
    power_component_h = math.pow(10,(elo_a - home_elo_adjusted)/400)
    # print(home_elo_adjusted - elo_a)
    power_component_a = math.pow(10,(home_elo_adjusted - elo_a)/400)

    denom_h = 1 + power_component_h
    denom_a = 1 + power_component_a

    num = 1

    win_prob_h = num / denom_h
    win_prob_a = num / denom_a

    return win_prob_h, win_prob_a

def k_calc(MOV,elo_diff):
    k = 20
    denom = 7.5 + 0.006*(elo_diff)
    num = math.pow(MOV+3,.8)
    multi = k*num/denom
    return multi

def update_elo(elo_h,elo_a,home_court,score_h,score_a):
    home_win_prob, away_win_prob = expected_win_prob(elo_h,elo_a,home_court)

    if score_h > score_a:
        mov = score_h-score_a
        s_h = 1
        s_a = 0
        elo_diff = elo_h-elo_a
    else:
        mov = score_a-score_h
        s_h = 0
        s_a = 1
        elo_diff = elo_a-elo_h
    k = k_calc(mov,elo_diff)

    updated_home_elo = elo_h + k*(s_h - home_win_prob) 
    updated_away_elo = elo_a + k*(s_a - away_win_prob)

    return updated_home_elo,updated_away_elo

In [ ]:
for index, row in netball_df.iterrows():
    if row['Team_A'] == 'Northern Mystics (Note 5)':
        print(index)
netball_df.at[42,'Team_A'] = 'Northern Mystics'

In [ ]:
def getEloBefore(team,elo_df,season,netball_df,date):
    if (team not in elo_df['Team_H'].values and team not in elo_df['Team_A'].values):
        return 1500
    for index, row in elo_df.iloc[::-1].iterrows():
        if row['Team_H'] == team:
            before_elo = row['H_elo_After']
            prev_season = int(netball_df[netball_df['Date'] < date]
                            [(netball_df['Team_H'] == team) | (netball_df['Team_A'] == team)].tail(1)['Season']
                            .iloc[0])
            break
        elif row['Team_A'] == team:
            before_elo = row['A_elo_After']
            prev_season = int(netball_df[netball_df['Date'] < date]
                [(netball_df['Team_H'] == team) | (netball_df['Team_A'] == team)].tail(1)['Season']
                .iloc[0])
            break
    if prev_season != season:
        return (0.75 * before_elo) + (0.25 * 1505)
    else:
        return before_elo
    

In [ ]:
elo_df = pd.DataFrame(columns=['GAME_ID','Team_H','Team_A','H_elo_Before','A_elo_Before','H_elo_After','A_elo_After'])
all_elo_df = pd.DataFrame(columns=['GAME_ID','Team','Elo','Season'])
for index, row in netball_df.iterrows():
    game_id = row['GameIDs']
    h_team = row['Team_H']
    h_score = row['Score_H']
    a_team = row['Team_A']
    a_score = row['Score_A']
    season = int(row['Season'])
    date = row['Date']

    elo_before_home = getEloBefore(h_team,elo_df,season,netball_df,date)
    elo_before_away = getEloBefore(a_team,elo_df,season,netball_df,date)

    elo_after_home, elo_after_away = update_elo(elo_before_home,elo_before_away,69,h_score,a_score)

    elo_df_new_row = {'GAME_ID':game_id,'Team_H':h_team,'Team_A':a_team,'H_elo_Before':elo_before_home,
                        'A_elo_Before':elo_before_away,'H_elo_After':elo_after_home,                                                'A_elo_After':elo_after_away}
                        
    all_elo_df_new_row_one = {'GAME_ID':game_id,'Team':h_team,'Elo':elo_before_home,'Date':date}
    all_elo_df_new_row_two = {'GAME_ID':game_id,'Team':a_team,'Elo':elo_before_away,'Date':date}

    elo_df = elo_df.append(elo_df_new_row, ignore_index=True)
    all_elo_df = all_elo_df.append(all_elo_df_new_row_one,ignore_index=True)
    all_elo_df = all_elo_df.append(all_elo_df_new_row_two,ignore_index=True)



In [ ]:
teams = netball_df['Team_H'].unique()

In [ ]:
import random
import seaborn as sns
all_elo_df['Elo'] = all_elo_df['Elo'].astype('float')

def runningMeanFast(x,N):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]
elos = all_elo_df['Elo'].values
plt.figure(figsize=(20,10))
samples_teams = random.sample(list(teams),k=3)
sns.set(font_scale=1.5)
sampled_data = all_elo_df.loc[all_elo_df['Team'].isin(samples_teams)]
sns.lineplot(data=sampled_data,x='Date',y='Elo',hue='Team')
plt.title('Elo over time')

In [ ]:
elo_df = elo_df.rename(columns={'GAME_ID':'GameIDs'})
elo_df.head()
netball_df.head()

In [ ]:
def get_average_stats(team, game_date, season_team_stats,n):
    prev_games_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['Team_H'] == team) |
    (season_team_stats['Team_A'] == team)].tail(n)
    
    if prev_games_df.empty:
        data = {'Score':0,'Against':0}
        team_df = pd.Series(data=data,index=['Score','Against'])
        return team_df    
    h_df = prev_games_df.iloc[:,[1,2]]
    h_df.columns = [x[:-2] for x in h_df.columns]
    a_df = prev_games_df.iloc[:,[3,4]]
    a_df.columns = [x[:-2] for x in a_df.columns]
    
    ah_df = pd.concat([h_df,a_df])
    opposition_scores_df = ah_df[ah_df['Team'] != team]
    opposition_scores_df.drop(columns='Team', inplace=True)
    opposition_scores_df = opposition_scores_df.rename(columns={'Score':'Against'})
    ah_df = ah_df[ah_df['Team'] == team]
    ah_df.drop(columns='Team', inplace=True)
    ah_df = pd.concat([ah_df,opposition_scores_df])
    return ah_df.mean()

average_stats = get_average_stats('Melbourne Vixens',netball_df['Date'][50],netball_df,10)
average_stats.index = ['H_last_10_AVG_'+ x for x in average_stats.index]


In [ ]:
#Used for testing
# pp = netball_df[netball_df['Date'] < netball_df['Date'][0]][(netball_df['Team_H'] == 'New South Wales Swifts') |
#     (netball_df['Team_A'] == 'New South Wales Swifts')].tail(10)
# if pp.empty:
#     data = {'Score':0,'Against':0}
#     team_df = pd.Series(data=data,index=['Score','Against'])
#     print(team_df.head())
#     print(type(team_df)) 
# h_df = pp.iloc[:,[1,2]]
# h_df.columns = [x[:-2] for x in h_df.columns]
# a_df = pp.iloc[:,[3,4]]
# a_df.columns = [x[:-2] for x in a_df.columns]
# ah_df = pd.concat([h_df,a_df])
# opposition_scores_df = ah_df[ah_df['Team'] != 'New South Wales Swifts']
# ah_df = ah_df[ah_df['Team'] == 'New South Wales Swifts']
# ah_df.drop(columns='Team', inplace=True)
# opposition_scores_df.drop(columns='Team', inplace=True)
# opposition_scores_df = opposition_scores_df.rename(columns={'Score':'Against'})

# ah_df = pd.concat([ah_df,opposition_scores_df])
# mean = ah_df.mean()
# print(mean)
# print(type(mean))


In [ ]:
recent_perforamnce = pd.DataFrame()

for season in netball_df['Season'].unique():
    temp_df = netball_df[netball_df['Season'] ==  season]
    n = 10

    season_recent_performance = pd.DataFrame()

    for index,row in temp_df.iterrows():
        game_id = row['GameIDs']
        home_team = row['Team_H']
        away_team = row['Team_A']
        game_date = row['Date']

        average_stats_home_team = get_average_stats(home_team, game_date, temp_df,n)
        average_stats_away_team = get_average_stats(away_team, game_date, temp_df,n)
        average_stats_home_team.index = ['H_last_10_AVG_'+ x for x in average_stats_home_team.index]
        average_stats_away_team.index = ['A_last_10_AVG_'+ x for x in average_stats_away_team.index]
        cols_H = [x for x in average_stats_home_team.index]
        cols_A = [ x for x in average_stats_away_team.index]
        cols_both = cols_H + cols_A
        new_row = pd.concat([average_stats_home_team,average_stats_away_team])
        new_row = new_row.T
        new_row['GameIDs'] = game_id
        season_recent_performance = season_recent_performance.append(new_row,ignore_index=True)
    recent_perforamnce = pd.concat([recent_perforamnce,season_recent_performance])
recent_perforamnce.head(15)

In [ ]:
final_team_stats = netball_df.merge(elo_df.drop(columns=['Team_H','Team_A']),on='GameIDs') \
                     .merge(recent_perforamnce.dropna(), on = 'GameIDs')

final_team_stats

In [ ]:
recent_perforamnce = recent_perforamnce.dropna()

In [ ]:
recent_perforamnce.isna().sum()

In [ ]:
recent_perforamnce[recent_perforamnce['GameIDs'] == 2]

In [ ]:
recent_perforamnce.reset_index()

In [ ]:
import math
def return_not_nan(a,b):
    if math.isnan(a):
        return b
    return a

In [ ]:
recent_perforamnce.head(1)['GameIDs']

In [ ]:
temp_row_id = 8888
temp_away_score = 0.0
temp_home_score = 0.0
count = 0
for index,row in recent_perforamnce.iterrows():
    game_id = row['GameIDs']
    home_score = row['H_last_10_AVG_Score']
    away_score = row['A_last_10_AVG_Score']
    if game_id == temp_row_id:
        new_home = return_not_nan(home_score,temp_home_score)
        new_away = return_not_nan(away_score,temp_away_score)
        recent_perforamnce['H_last_10_AVG_Score'][index] = new_home
        recent_perforamnce['A_last_10_AVG_Score'][index] = new_away
    temp_row_id = game_id
    temp_home_score = home_score
    temp_away_score = away_score


In [ ]:
recent_perforamnce.dropna()

In [ ]:
final_team_stats.head()

In [ ]:
final_team_stats['home_win'] = (final_team_stats['Score_H'] > final_team_stats['Score_A']).astype(int)
final_team_stats.head()

In [ ]:
final_team_stats.to_csv('final_team_stats.csv',index=False)